In [8]:
import pandas as pd
file_name = "data/virtual_hospital/orginal_VP_data/physician.csv"
data = pd.read_csv(file_name)

In [5]:
def contains_subjective(text):
    subjective_keywords = ["reports", "complains", "symptoms", "history of"]
    return any(keyword in text.lower() for keyword in subjective_keywords)

def contains_objective(text):
    objective_keywords = ["vital signs", "physical examination", "labs", "imaging", "findings"]
    return any(keyword in text.lower() for keyword in objective_keywords)

def contains_assessment(text):
    objective_keywords = ["assessment"]
    return any(keyword in text.lower() for keyword in objective_keywords)

def contains_plan(text):
    objective_keywords = ["plan"]
    return any(keyword in text.lower() for keyword in objective_keywords)

def check_condition(text):
    if contains_subjective(text) and contains_objective(text):
        return True
    return False


In [6]:
# 直接根据assessment and plan来分割，前面是s_and_o，后面是a_and_p
def split_assessment_plan(text):
    match = re.search(r'assessment and plan', text, re.IGNORECASE)
    
    if match:
        split_index = match.start()
        s_and_o = text[:split_index].strip()
        a_and_p = text[split_index:].strip()
        return s_and_o, a_and_p
    else:
        return None, None

In [9]:
import pandas as pd
import json
import re
result_list = []

# grouped by SUBJECT_ID
grouped = data.groupby('SUBJECT_ID')
counter = 0
for subject_id, group in grouped:
    # sorted by row_id
    sorted_group = group.sort_values(by='ROW_ID', ascending=False)
    
    for index, row in sorted_group.iterrows():
        if check_condition(row['TEXT']):
            row_data = row.to_dict()
            
        
            s_and_o, a_and_p = split_assessment_plan(row['TEXT'])
            
            if s_and_o and a_and_p:
                row_data['s_and_o'] = s_and_o
                row_data['a_and_p'] = a_and_p
                result_list.append(row_data)
                counter += 1
                break
            else:
                continue
print(counter)

with open('data/virtual_hospital/MIMIC_unique_notes.json', 'w') as json_file:
    json.dump(result_list, json_file, indent=4)

print("unique_notes.json is done")
# print(len(result_list))

# # store into a json file
# with open('unique_notes.json', 'w') as json_file:
#     json.dump(result_list, json_file, indent=4)

# print("unique_notes.json is done")

# 先以subject_id分组，然后对其row_id进行从近到远排序，对应的就是降序；然后从上到下遍历，直到取到满足soap condition并且能分割assessment and plan的note
# 这样结束之后有5074条满足

5074
unique_notes.json is done


In [4]:
import json
with open("data/virtual_hospital/MIMIC_unique_notes.json", "r") as f:
    physician_note = json.load(f)
for v in physician_note:
    print(v.keys())
    print(v['SUBJECT_ID'])
    print(v['ROW_ID'])
    break
# print(physician_note)

dict_keys(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME', 'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'ISERROR', 'TEXT', 's_and_o', 'a_and_p'])
109
385223
